### Configure Azure ML Credentials & Compute Target

In [71]:
#import required libraries
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
from dotenv import load_dotenv
import os

#Enter details of your Azure Machine Learning workspace
load_dotenv()
subscription_id = os.environ.get('SUBSCRIPTION_ID')
resource_group = os.environ.get('RESOURCE_GROUP')
workspace = os.environ.get('WORKSPACE_NAME')

#connect to the workspace
ml_client = MLClient(DefaultAzureCredential(), subscription_id, resource_group, workspace)

In [68]:
from azure.ai.ml.entities import AmlCompute

# specify aml compute name.
cpu_compute_target = "sydney-compute-cpu"

try:
    ml_client.compute.get(cpu_compute_target)
except Exception:
    print("Creating a new cpu compute target...")
    compute = AmlCompute(
        name=cpu_compute_target, size="STANDARD_D2_V2", min_instances=0, max_instances=4
    )
    ml_client.compute.begin_create_or_update(compute).result()

### Submit the Training Job to Azure ML Compute Resource

In [69]:
from azure.ai.ml import command

# define the command
command_job = command(
    code="./",
    command="python train.py",
    environment="AzureML-acpt-pytorch-2.2-cuda12.1@latest",
    compute=cpu_compute_target,
    experiment_name="train-on-cpu",
    display_name="train-image-classifier"
)

# submit the command
returned_job = ml_client.jobs.create_or_update(command_job)
# get a URL for the status of the job
returned_job.studio_url

Your file exceeds 100 MB. If you experience low speeds, latency, or broken connections, we recommend using the AzCopyv10 tool for this file transfer.

Example: azcopy copy '/Users/sydneylester/Projects/elephant-image-classification' 'https://sydney4772397004.blob.core.windows.net/1b0adfc3-11ad-451c-806a-68d87a4f8bb3-127cy4h9yrf75k1fia3qh96qvv/elephant-image-classification' 

See https://docs.microsoft.com/azure/storage/common/storage-use-azcopy-v10 for more information.
Uploading elephant-image-classification (417.97 MBs):   0%|          | 66288/417966820 [00:28<50:20:32, 2305.89it/s]


KeyboardInterrupt: 

### Register Model & Save Locally

In [ ]:
from azure.ai.ml.entities import Model
from azure.ai.ml.constants import AssetTypes

# Get the completed job
job_details = ml_client.jobs.get(returned_job.name)

# Register the model from the job's outputs
registered_model = ml_client.models.create_or_update(
    Model(
        path=f"{job_details.outputs['artifacts']}/outputs/elephant_classifier_resnet50.pth",  # Job output path
        name="image-classifier",
        description="Trained image classifier",
    )
)

ml_client.models.download(registered_model, "elephant_classifier_resnet50.pth")
